# Prepare Open AI client object

In [1]:
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
client

# Prepare documents for information retrieval

In [5]:
import json

In [6]:
with open("documents-with-ids.json", "rt") as f_in:
    documents = json.load(f_in)

# Create Vector Embeddings

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Elastic Search

In [11]:
from elasticsearch import Elasticsearch

In [12]:
es_client = Elasticsearch("http://localhost:9200")

## Create index

In [13]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine",
            },
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine",
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine",
            },
        }
    },
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Encode documents to vectors and data to index

In [14]:
from tqdm.auto import tqdm

In [15]:
for doc in tqdm(documents):
    question = doc["question"]
    text = doc["text"]
    qt = question + " " + text

    doc["question_vector"] = model.encode(question)
    doc["text_vector"] = model.encode(text)
    doc["question_text_vector"] = model.encode(qt)

  0%|          | 0/948 [00:00<?, ?it/s]

In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Evaluate Embedding Retrieval

In [22]:
question = "I just discovered the course. Can I still join it?"

In [25]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {"term": {"course": course}},
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"],
    }

    es_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [26]:
def question_vector_knn(q):
    question = q["question"]
    course = q["course"]

    v_q = model.encode(question)

    return elastic_search_knn("question_vector", v_q, course)

In [32]:
def text_vector_knn(q):
    answer = q["question"]
    course = q["course"]

    v_q = model.encode(answer)

    return elastic_search_knn("text_vector", v_q, course)

In [34]:
def question_text_vector_knn(q):
    answer = q["question"]
    course = q["course"]

    v_q = model.encode(answer)

    return elastic_search_knn("question_text_vector", v_q, course)

## Evaluate if elastic search returns the correct question

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [18]:
import pandas as pd

df_ground_truth = pd.read_csv("ground-truth-data.csv")
ground_truth = df_ground_truth.to_dict(orient="records")

In [21]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q["document"]
        results = search_function(q)
        relevance = [d["id"] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        "hit_rate": hit_rate(relevance_total),
        "mrr": mrr(relevance_total),
    }

##### Compare with text only retrieval:

ES: {'hit_rate': 0.7456709956709957, 'mrr': 0.6078354978354981}

minsearch: {'hit_rate': 0.783982683982684, 'mrr': 0.6744264069264069}

### Search using question only

In [28]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/4620 [00:00<?, ?it/s]

{'hit_rate': 0.7640692640692641, 'mrr': 0.6638780663780672}

**Comment:** better than ES using only text but minsearch still performs better!

### Search using text only

In [33]:
evaluate(ground_truth, text_vector_knn)

  0%|          | 0/4620 [00:00<?, ?it/s]

{'hit_rate': 0.8162337662337662, 'mrr': 0.6930735930735934}

**Comment:** better than all retrievals tested before!

### Search using text and question

In [35]:
evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/4620 [00:00<?, ?it/s]

{'hit_rate': 0.9075757575757576, 'mrr': 0.8205230880230889}

**Comment:** better than all retrievals tested before! --> BEST RESULT

### Combine everything

In [37]:
def elastic_search_knn_combined(vector, course):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {"term": {"course": course}},
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {"query_vector": vector},
                            },
                        }
                    }
                ],
                "filter": {"term": {"course": course}},
            }
        },
        "_source": ["text", "section", "question", "course", "id"],
    }

    es_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs


def vector_combined_knn(q):
    question = q["question"]
    course = q["course"]

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)


evaluate(ground_truth, vector_combined_knn)

  0%|          | 0/4620 [00:00<?, ?it/s]

{'hit_rate': 0.8943722943722944, 'mrr': 0.799790764790766}

# Homework

In [38]:
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [146]:
user_question = "I just discovered the course. Can I still join it?"
vector_question = model.encode(user_question)

In [147]:
vector_question.shape

(768,)

In [41]:
vector_question[0]

0.07822261

This vector is normalized:

In [55]:
vector_question.dot(vector_question)

1.0

## Q1: 0.07822261

In [42]:
import requests

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/documents-with-ids.json"
docs_url = f"{base_url}/{relative_url}?raw=1"
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [46]:
filtered_documents = []

for doc in documents:
    if doc["course"] == "machine-learning-zoomcamp":
        filtered_documents.append(doc)

In [48]:
len(filtered_documents)

375

In [66]:
embeddings = []

for doc in filtered_documents:

    qa_text = "{question} {text}".format(**doc)

    embeddings.append(model.encode(qa_text))

In [67]:
import numpy as np

In [68]:
X = np.array(embeddings)

In [69]:
X.shape

(375, 768)

## Q2: (375, 768)

In [70]:
X.dot(vector_question).max()

0.6506574

## Q3: 0.65

In [133]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, query, num_results=5):

        v_query = model.encode(query)
        scores = self.embeddings.dot(v_query)
        idx = np.argpartition(-scores, num_results)[:num_results]
        return [self.documents[i] for i in idx]

In [131]:
search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_results = search_engine.search(user_question, num_results=5)

search_results

In [91]:
import pandas as pd

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/ground-truth-data.csv"
ground_truth_url = f"{base_url}/{relative_url}?raw=1"

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == "machine-learning-zoomcamp"]
ground_truth = df_ground_truth.to_dict(orient="records")

In [135]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q["document"]
        results = search_function(q["question"])
        relevance = [d["id"] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        "hit_rate": hit_rate(relevance_total),
        "mrr": mrr(relevance_total),
    }

In [97]:
evaluate(ground_truth, search_engine.search)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.5639526411657593}

## Q4: 0.93

In [145]:
X[0].shape

(768,)

Add all embeddings to documents dictionary

In [148]:
for index, doc in enumerate(filtered_documents):
    doc["question_text_vector"] = X[index]

In [149]:
es_client = Elasticsearch("http://localhost:9200")

In [150]:
es_client.info()

ObjectApiResponse({'name': 'd9d355a53607', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'mykHg1ngRYmjQQO5Lei-og', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [152]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine",
            },
        }
    },
}

index_name = "course-questions-homework"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework'})

In [153]:
for doc in tqdm(filtered_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [154]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"],
    }

    es_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [158]:
elastic_search_knn("question_text_vector", vector_question)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

## Q5: ee58a693

In [155]:
def question_text_vector_knn(q):

    v_q = model.encode(q)

    return elastic_search_knn("question_text_vector", v_q)

In [156]:
evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

## Q6: 0.93